In [24]:
import csv

def agg(m, price_point):
    m_stats = m.setdefault('stats', {})
    m_agg = m_stats.setdefault('total', 0)
    m_count = m_stats.setdefault('count', 0)
    m_dp = m_stats.setdefault('dp', [])
    
    m_stats['total'] = m_agg + price_point
    m_stats['count'] = m_count + 1
    m_stats['dp'].append(price_point)

    
def calc(m):
    if type(m) != dict or 'stats' not in m:
        return
    m_stats = m['stats']
    m_stats['avg'] = m_stats['total'] / m_stats['count']
    
    m_stats['dp'].sort()
    m_stats['median'] = m_stats['dp'][int(len(m_stats['dp']) * 0.5 - 1)]
    m_stats['p90'] = m_stats['dp'][int(len(m_stats['dp']) * 0.9 - 1)]
    
    m_stats['dp'] = []
    
    
matrix = {}
with open('training.csv', 'r') as f_in:
    reader = csv.DictReader(f_in)
    for row in reader:
        
        # skip empty cities, property types, and prices
        if not row['CITY'] or not row['PROPERTY_TYPE'] or not row['PRICE']:
            continue
        
        price_point = int(row['PRICE'])
        city_level = matrix.setdefault(row['CITY'], {})
        p_type_level = city_level.setdefault(row['PROPERTY_TYPE'], {})
        bb_level = p_type_level.setdefault(str(row['BEDS']) + "_" + str(row['BATHS']), {})
        
        agg(city_level, price_point)
        agg(p_type_level, price_point)
        agg(bb_level, price_point)
        
    
    for city in matrix:
        calc(matrix[city])
        for p_type in matrix[city]:
            calc(matrix[city][p_type])
            for bb in matrix[city][p_type]:
                calc(matrix[city][p_type][bb])    

In [54]:
def query(in_city, # required
          in_p_type=None,  # optional
          n_beds=None,  # optional
          n_baths=None # optional
         ):
    
    in_city = in_city.upper().strip()
    if in_city not in matrix:
        raise Exception("City {} not in available cities: {}".format(in_city, matrix.keys()))
    city = matrix[in_city]
    
    if not in_p_type:
        return city['stats']
    
    in_p_type = in_p_type.upper().strip()
    if in_p_type not in matrix[in_city]:
        raise Exception("Property type {} not in available ptypes: {}".format(in_p_type, matrix[in_city].keys()))
        
    if not n_beds or not n_baths:
        return matrix[in_city][in_p_type]['stats']
    
    return matrix[in_city][in_p_type].get(str(n_beds) + "_" + str(float(n_baths)), "No data exists for {} beds and {} baths. Available: {}".format(n_beds, n_baths, matrix[in_city][in_p_type].keys()))
    
    
query(in_city='redwood city', in_p_type='SINGLE_FAMILY_HOME', n_beds=2, n_baths=1)

{'stats': {'total': 14208000,
  'count': 9,
  'dp': [],
  'avg': 1578666.6666666667,
  'median': 1360000,
  'p90': 1900000}}